# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/) - само аналитическое разложение вам реализовывать НЕ НУЖНО

In [1]:
import random
import time
import torch
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

import tensorly as tl
tl.set_backend('pytorch')
from tensorly.decomposition import tucker
from tensorly.tucker_tensor import tucker_to_tensor

torch.manual_seed(0)

## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [2]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=(10, 20, 15)):
    # data - тензор с заданной размерностью
    # U - список матриц
    # G - ядро разложения
    G = torch.randint(10, (r[0], r[1], r[2]), requires_grad=True, dtype=torch.double)
    U = [torch.randint(10, (size[i], r[i]), requires_grad=True, dtype=torch.double) for i in range(3)]

    data = G
    
    data = torch.tensordot(U[0], data, dims=([1], [0]))
    
    data = torch.tensordot(U[1], data.permute([1, 0, 2]), dims=([1], [0]))
    data = data.permute([1, 0, 2])
    
    data = torch.tensordot(U[2], data.permute([2, 1, 0]), dims=([1], [0]))
    data = data.permute([2, 1, 0])

    return data, U, G

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [3]:
size = (100, 200, 300)
r = (10, 20, 30)

data, U, G = get_tensor(size, r)
data.shape, [u.shape for u in U], G.shape

(torch.Size([100, 200, 300]),
 [torch.Size([100, 10]), torch.Size([200, 20]), torch.Size([300, 30])],
 torch.Size([10, 20, 30]))

In [4]:
noise = torch.randint(10, (data.shape)) * 1e-2
data += noise

Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ:
+ Разложение Такера ищет простые структуры в данных. Если данные случайные и структуры нет, разложение бесполезно, результат будет почти таким же, как и исходные данные.
+ Градиентный спуск плохо работает с очень сложными данными, у которых множество локальных минимумов. А случайные данные именно такие.
+ Добавление шума помогает найти глобальный минимум среди множества других.

## 2 Реализуйте метод для восстановления тензора по разложению

In [5]:
# Функция, восстанавливающая тензор по ядру и матрицам
def repair_tensor(G_, U):
    # data - восстановленный тензор из матриц и ядра
    # U - список матриц
    # G_ - ядро разложения
    data = G_
    for i, U_i in enumerate(U):
        permutation = list(range(data.dim()))
        permutation[0], permutation[i] = permutation[i], permutation[0]
        data = torch.tensordot(U_i, data.permute(permutation), dims=([1], [0])).permute(permutation)
    return data

In [6]:
data_test, U_test, G_test = get_tensor()
reconstructed_data = repair_tensor(G_test, U_test)
print(torch.allclose(data_test, reconstructed_data))

r = (15, 20, 25)
_, U, G = get_tensor(size=(150, 200, 250), r=r)

data = repair_tensor(G, U)
print('\ndata:', data.shape)

data_p = tucker_to_tensor((G, U))
print('data_p:', data_p.shape)

print('\nMSE:', mean_squared_error(data.detach().numpy().flatten(), data_p.detach().numpy().flatten()))
print((data==data_p).all())

True

data: torch.Size([150, 200, 250])
data_p: torch.Size([150, 200, 250])

MSE: 0.0
tensor(True)


## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [7]:
G_bib, U_bib = tucker(data, r)

Не забудьте померить ошибку разложения по метрике MSE

In [8]:
data_bib = tucker_to_tensor((G_bib, U_bib))
print("MSE:", mean_squared_error(data.detach().numpy().flatten(), data_bib.detach().numpy().flatten()))

MSE: 2.1962010929834375e-17


## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [9]:
import math
import torch
from torch.optim.optimizer import Optimizer
from torch.optim import SGD


class Opt(Optimizer):

    def __init__(self, params, lr=1e-3):
        self.lr = lr
        defaults = dict(lr=self.lr)
        super().__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for param in group['params']:
                if param.grad is None:
                    continue

                grad = param.grad.data
                if torch.isnan(grad).any() or torch.isinf(grad).any():
                    print("Warning: NaN or Inf gradient detected for parameter:", param)
                    continue  

                param.data.add_(-group['lr'], grad)

        return loss

### 4.2 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [10]:
class TuckerModel(nn.Module):
    def __init__(self, factor_matrices, core_tensor):
        super().__init__()
        self.factor_matrices = nn.ParameterList([nn.Parameter(u) for u in factor_matrices])
        self.core_tensor = nn.Parameter(core_tensor)

    def forward(self):
        return repair_tensor(self.core_tensor, self.factor_matrices)

In [11]:
r = (10, 15, 25)
size = (150, 200, 250)

data_train, U, G = get_tensor(size = size, r = r)
noise = torch.randint(10, (data_train.shape))
data_train += noise
data_train = data_train.detach()

In [14]:
def train_model(data_train, U, G, num_epochs=100, verbose=True, log_frequency=10, lr=1e-10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    data_train = data_train.to(device)
    U = [u.to(device) for u in U]
    G = G.to(device)

    model = TuckerModel(U, G).to(device) 
    criterion = nn.MSELoss()
    optimizer = Opt(model.parameters(), lr = lr)

    loss_history = []
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        data_predicted = model.forward()
        loss = criterion(data_predicted, data_train)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())

        if verbose and epoch % log_frequency == 0:
            print(f"--------Epoch {epoch}--------")
            print(f"Loss {loss.item()}")
            print(f"-----------------------\n")

    return model, loss_history

In [15]:
model, loss_history = train_model(data_train, U, G)

Using device: cpu
--------Epoch 0--------
Loss 8.57948525944512
-----------------------

--------Epoch 10--------
Loss 8.535723800854878
-----------------------

--------Epoch 20--------
Loss 8.501861575731894
-----------------------

--------Epoch 30--------
Loss 8.4746523964
-----------------------

--------Epoch 40--------
Loss 8.452184111308302
-----------------------

--------Epoch 50--------
Loss 8.433267918007859
-----------------------

--------Epoch 60--------
Loss 8.41712106214531
-----------------------

--------Epoch 70--------
Loss 8.403198826092229
-----------------------

--------Epoch 80--------
Loss 8.391103235147732
-----------------------

--------Epoch 90--------
Loss 8.38053174455751
-----------------------



## 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [16]:
train_data = []

for _ in range(20):
    data_train, U, G = get_tensor(size=size, r=r)
    noise = torch.randint(10, (data_train.shape))*1e-2
    data_train += noise
    data_train = data_train.detach()
    train_data.append((data_train, U, G))

In [17]:
losses_grad = []

time1 = time.time()
for i in range(20):
    model, loss_history = train_model(*train_data[i], verbose=False, num_epochs=30)
    losses_grad.append(loss_history[-1])
time2 = time.time() - time1

Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu
Using device: cpu


In [18]:
print("loss:", sum(losses_grad) / len(losses_grad), "\ntime:", np.round(time2, 5))

loss: 0.0011473455898549391 
time: 95.94947


In [19]:
losses_lib = []

time1 = time.time()
for i in range(20):
    data, U, G = train_data[i]
    data_p = tucker_to_tensor((G, U))
    losses_lib.append(mean_squared_error(data.detach().numpy().flatten(), data_p.detach().numpy().flatten()))
time2 = time.time() - time1

In [20]:
print("loss:", sum(losses_lib) / len(losses_lib), "\ntime:", np.round(time2, 3))

loss: 0.0011295887886070004 
time: 1.985
